In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

from sklearn.metrics import r2_score

In [94]:
df = pd.read_csv('test_filtered_df.csv')
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,basement,year,Renovated,age_when_sold
0,221900.0,3,1.00,1180,5650,1.0,0,3,7,1180,0.0,1955,0,1340,5650,0,2014,0,59
1,538000.0,3,2.25,2570,7242,2.0,0,3,7,2170,400.0,1951,1991,1690,7639,1,2014,1,63
2,180000.0,2,1.00,770,10000,1.0,0,3,6,770,0.0,1933,0,2720,8062,0,2015,0,82
3,604000.0,4,3.00,1960,5000,1.0,0,5,7,1050,910.0,1965,0,1360,5000,1,2014,0,49
4,510000.0,3,2.00,1680,8080,1.0,0,3,8,1680,0.0,1987,0,1800,7503,0,2015,0,28


In [95]:
num = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']

In [96]:
for feat in num:
    df[feat] = np.log(df[feat])

C:\Users\damni\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [97]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,basement,year,Renovated,age_when_sold
0,221900.0,3,1.00,7.073270,8.639411,1.0,0,3,7,7.073270,-inf,1955,0,7.200425,8.639411,0,2014,0,59
1,538000.0,3,2.25,7.851661,8.887653,2.0,0,3,7,7.682482,5.991465,1951,1991,7.432484,8.941022,1,2014,1,63
2,180000.0,2,1.00,6.646391,9.210340,1.0,0,3,6,6.646391,-inf,1933,0,7.908387,8.994917,0,2015,0,82
3,604000.0,4,3.00,7.580700,8.517193,1.0,0,5,7,6.956545,6.813445,1965,0,7.215240,8.517193,1,2014,0,49
4,510000.0,3,2.00,7.426549,8.997147,1.0,0,3,8,7.426549,-inf,1987,0,7.495542,8.923058,0,2015,0,28


In [98]:
sk_ols = LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)

In [99]:
rfe = RFE(sk_ols, n_features_to_select=5, step=1, verbose=2)

In [100]:
X_all = df.drop(columns=['price', 'yr_renovated', 'yr_built', 'sqft_basement'])
rfe.fit(X_all, df['price'])

Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1,
                               normalize=False),
    n_features_to_select=5, step=1, verbose=2)

In [101]:
new_cols = X_all.columns[rfe.support_]

In [102]:
X_new = df[new_cols]
X_new.head()

,view,grade,sqft_living15,basement,Renovated
0,0,7,7.200425,0,0
1,0,7,7.432484,1,1
2,0,6,7.908387,0,0
3,0,7,7.215240,1,0
4,0,8,7.495542,0,0


In [87]:
def make_ols_sklearn(X, y, test_size=0.20, fit_intercept=False, standardize=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    if standardize:
        ss = StandardScaler()
        ss.fit(X_train)
        X_train = ss.transform(X_train)
        X_test = ss.transform(X_test)
    ols = LinearRegression(fit_intercept=fit_intercept, normalize=False)
    ols.fit(X_train, y_train)
    train_score = ols.score(X_train, y_train)
    test_score = ols.score(X_test, y_test)
    print(f"train score = {train_score}")
    print(f"test score = {test_score}")
    return ols

In [103]:
make_ols_sklearn(X_new, df['price'])

train score = 0.4307413743398735
test score = 0.410103251884293


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [104]:
cv_scores = cross_val_score(sk_ols, X_new, df['price'], scoring='r2', cv=10, n_jobs=-1) 
cv_scores

array([0.42738539, 0.42592283, 0.40556966, 0.40503311, 0.431014  ,
       0.42176587, 0.43578731, 0.44268948, 0.43651462, 0.40361647])

In [90]:
target ='price'

In [105]:
train, test = train_test_split(df)

In [106]:
predictors = '+'.join(new_cols)
formula = target + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     2812.
Date:                Wed, 03 Jun 2020   Prob (F-statistic):               0.00
Time:                        14:44:03   Log-Likelihood:            -2.0816e+05
No. Observations:               15354   AIC:                         4.163e+05
Df Residuals:                   15348   BIC:                         4.164e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.641e+06    4.1e+04    -40.056      0.000   -1.72e+06   -1.56e+06
view           5.056e+04   2384.396     21.204      0.000    4.59e+04    5.52e+04
grade          1.268e+05   2018.002     62.830      0.000    1.23e+05    1.31e+05
sqft_living15  1.515e+05   6527.823     23.211      0.000    1.39e+05    1.64e+05
basement       7.279e+04   3153.531     23.083      0.000    6.66e+04     7.9e+04
Renovated       1.42e+05   8392.702     16.914      0.000    1.26e+05    1.58e+05
==============================================================================
Omnibus:                     4154.110   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15930.161
Skew:                           1.309   Prob(JB):                         0.00
Kurtosis:                       7.248   Cond. No.                         296.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""